In [2]:
import re
import pandas as pd
import numpy as np

#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import os
import os.path

path = "/content/sample_data/" 
os.chdir(path)

In [16]:
# This is the 60k tweets selected from the Carmen processed data
df = pd.read_csv('NLP_train.csv')
df.head()

,Unnamed: 0,id,text,author_id,public_metrics,created_at,in_reply_to_user_id,conversation_id,reply_settings,source,referenced_tweets,context_annotations,entities,user,user_name,city,state,county,country,baseline_label
0,0,8898041343901696,She was sick last week and now again last nigh...,165742597,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2010-11-28T15:00:33.000Z,NaN,8898041343901696,everyone,Twitter Web Client,NaN,NaN,"{'hashtags': [{'start': 99, 'end': 104, 'tag':...","{'location': 'PA', 'created_at': '2010-07-12T1...",GFfoodsROCK,NaN,Pennsylvania,NaN,United States,True
1,1,10423674930733056,Anyone take tindamax for #Lyme DD takes this l...,165742597,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2010-12-02T20:02:52.000Z,NaN,10423674930733056,everyone,Twitter Web Client,NaN,NaN,"{'hashtags': [{'start': 25, 'end': 30, 'tag': ...","{'location': 'PA', 'created_at': '2010-07-12T1...",GFfoodsROCK,NaN,Pennsylvania,NaN,United States,True
2,2,7647964877,Time for Lyme gives research grant to Dr. Robe...,34282571,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2010-01-11T23:45:24.000Z,NaN,7647964877,everyone,Twitter Web Client,NaN,NaN,"{'annotations': [{'start': 9, 'end': 12, 'prob...","{'username': 'Kulambo', 'public_metrics': {'fo...",Kulambo,NaN,NaN,NaN,United States,False
3,3,19800587398815744,@IkenCEO @filemot Hv just started novel (set i...,74001901,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2010-12-28T17:03:22.000Z,49366829.0,19789870159372289,everyone,Twitter Web Client,"[{'type': 'replied_to', 'id': '197898701593722...",NaN,"{'annotations': [{'start': 48, 'end': 57, 'pro...","{'name': 'Rochelle Almeida', 'location': 'Sout...",southportgal,NaN,Connecticut,NaN,United States,False
4,4,12513156315,"Dog Blogged:: Old Lyme Disease - Oh yeah, I've...",20862522,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2010-04-20T12:26:57.000Z,NaN,12513156315,everyone,Twitter Web Client,NaN,NaN,NaN,"{'location': 'CT USA', 'created_at': '2009-02-...",ScouttheDog,NaN,Connecticut,NaN,United States,True


In [17]:
# Source/reference- https://medium.com/analytics-vidhya/nlp-tutorial-for-text-classification-in-python-8f19cd17b49e

#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
    
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
    
df['clean_text'] = df['text'].apply(lambda x: finalpreprocess(x))
df.head()

In [ ]:
# Create baseline dataframe with tweets randomly labelled with 0 (non-Lyme) and 1 (Lyme)
data = np.random.randint(0,2,size=len(df))
df['random_label'] = data
df.head()

In [20]:
# Source/reference- https://medium.com/analytics-vidhya/nlp-tutorial-for-text-classification-in-python-8f19cd17b49e

#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['baseline_label'], test_size=0.2, shuffle=True)

#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['clean_text']]
model = Word2Vec(df['clean_text_tok'],min_count=1)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))     
modelw = MeanEmbeddingVectorizer(w2v)

# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

In [22]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)

#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

       False       0.92      0.97      0.95      3462
        True       0.99      0.97      0.98      8538

    accuracy                           0.97     12000
   macro avg       0.96      0.97      0.96     12000
weighted avg       0.97      0.97      0.97     12000

Confusion Matrix: [[3360  102]
 [ 276 8262]]
AUC: 0.992621713320502


In [ ]:
# Test Model with 6000 Ground Truth Labeled Tweets with Geo Tags

In [ ]:
# Use Model to Classify 27994 Tweets with Geo Tags

In [ ]:
# Correlate the 27994 Classified Tweets to CDC Data

In [ ]:
# Plan B: Isolate Tweets that Confirm the person has Lyme or No Lyme. Then repeat the prior steps